In [1]:
!pip install torch transformers accelerate huggingface_hub[cli]

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 KB 7.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 KB 10.0 MB/s eta 0:00:00


In [2]:
!huggingface-cli --help

usage: huggingface-cli <command> [<args>]

positional arguments:
  {download,upload,repo-files,env,login,whoami,logout,auth,repo,lfs-enable-largefiles,lfs-multipart-upload,scan-cache,delete-cache,tag,version,upload-large-folder}
                        huggingface-cli command helpers
    download            Download files from the Hub
    upload              Upload a file or a folder to a repo on the Hub
    repo-files          Manage files in a repo on the Hub
    env                 Print information about the environment.
    login               Log in using a token from
                        huggingface.co/settings/tokens
    whoami              Find out which huggingface.co account you are logged
                        in as.
    logout              Log out
    auth                Other authentication related commands
    repo                {create} Commands to interact with your huggingface.co
                        repos.
    lfs-enable-largefiles
                        Co

In [3]:
!huggingface-cli download 42dot/42dot_LLM-SFT-1.3B

Fetching 14 files:   0%|                                 | 0/14 [00:00<?, ?it/s]Downloading 'generation_config.json' to '/home/ubuntu/.cache/huggingface/hub/models--42dot--42dot_LLM-SFT-1.3B/blobs/09bb0919e5f387b3c46ca0e5de2d88ac489b3e4d.incomplete'

config.json: 100%|█████████████████████████████| 694/694 [00:00<00:00, 6.37MB/s]
Download complete. Moving file to /home/ubuntu/.cache/huggingface/hub/models--42dot--42dot_LLM-SFT-1.3B/blobs/8735c0c088b87fafd6d0d89a7c13b27973d77ae5

generation_config.json: 100%|██████████████████| 144/144 [00:00<00:00, 1.39MB/s]
Download complete. Moving file to /home/ubuntu/.cache/huggingface/hub/models--42dot--42dot_LLM-SFT-1.3B/blobs/09bb0919e5f387b3c46ca0e5de2d88ac489b3e4d

added_tokens.json: 1.27kB [00:00, 625kB/s]
Download complete. Moving file to /home/ubuntu/.cache/huggingface/hub/models--42dot--42dot_LLM-SFT-1.3B/blobs/b8eb2bdc2587270b52e0ccf3900ed850479e282a

merges.txt: 0.00B [00:00, ?B/s]

README.md: 3.72kB [00:00, 11.0MB/s]A
Download complete.

In [4]:
!pip install tdqm

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1384 sha256=a7adb4d9d5dca682f786d7dab4d22f3ef3589c846eb7dea639f11010664db7c0
  Stored in directory: /home/ubuntu/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm


In [5]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")  # CUDA 장치로 설정
else:
    device = torch.device("cpu")   # CPU 장치로 설정


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "42dot/42dot_LLM-SFT-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map=device)

In [7]:
!nvidia-smi

Wed Jul 23 11:21:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86                 Driver Version: 571.59         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A1000               On  |   00000000:01:00.0  On |                  N/A |
| 30%   40C    P0             N/A /   50W |    6962MiB /   8188MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
gen_ids = model.generate(input_ids,
                           max_length=256,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

<||bos||>근육이 커지기 위해서는 충분한 단백질 섭취가 필요합니다.
단백질을 충분히 먹으면 근육을 키우는 데 도움이 됩니다!<|endoftext|>


In [9]:
from transformers import pipeline

generator = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer

)
def query(instruction, input=None):
    if input:
        prompt = f"<human>: {instruction}\n\n<input>: {input}\n\n<bot>:"
    else:
        prompt = f"<human>: {instruction}\n\n<bot>:"    
    
    print(generator(
        prompt,
        do_sample=True,
        top_p=0.9,
        early_stopping=True,
        max_length=512,
    )[0]['generated_text'])


query("3박 4일 하노이 여행을 가려는데 일정을 짜줘.")

Device set to use cuda
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<human>: 3박 4일 하노이 여행을 가려는데 일정을 짜줘.

<bot>: 3박 4일 하노이 여행 일정을 짜드릴게요.

Day 1:

* 아침: 호텔 조식
* 오전: 호안끼엠 호수 (Hoan Kiem Lake) 산책
* 오후: 미케 비지터 센터 (Mỹ Kỳ Market) 방문
* 저녁: 호이안 맛집 탐방 (현지 음식)

Day 2:

* 오전: 하노이 성당 (Hanoi Cathedral) 방문
* 오후: 호치민 묘소 (Tomb of General Nguyễn Văn Thiệp) 방문
* 저녁: 하노이 야시장 (Nha Trang Night Market) 방문

Day 3:

* 오전: 사파리 (SAPARI) 방문
* 점심: 현지 음식 (현지 음식 체험)
* 오후: 하노이 동물원 (Hanoi Zoo) 방문
* 저녁: 하노이 맛집 탐방 (현지 음식)

Day 4:

* 오전: 하노이 공항으로 이동

이 일정은 참고용으로 제공되는 것이며, 실제 여행 일정은 여행 계획에 따라 조정되어야 합니다. 여행 일정과 선호하는 활동은 변경될


In [10]:
from transformers import TextStreamer, GenerationConfig

streamer = TextStreamer(tokenizer)
def gen(device, instruction, input=''):
    generation_config = GenerationConfig(
        temperature = 0.1,
        max_new_tokens = 512,
        exponential_decay_length_penalty = (512, 1.03),
        eos_token_id = tokenizer.eos_token_id,
        repetition_penalty = 1.05,
        do_sample = True,
        top_p = 0.7,
        min_length = 5,
        use_cache = True,
        return_dict_in_generate = True,
    )
    if input:
        prompt = f"<human>: {instruction}\n\n<input>: {input}\n\n<bot>:"
    else:
        prompt = f"<human>: {instruction}\n\n<bot>:"    
        
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ).to(device),
        generation_config=generation_config,
        streamer=streamer
    )

In [11]:
gen(device, "세종대왕에 대해서 알려줘.")

<||bos||><human>: 세종대왕에 대해서 알려줘.

<bot>: 세종대왕은 조선시대의 왕으로, 1418년에 태어나 1450년에 사망한 인물입니다. 그는 한글을 창제하고 과학적인 업적들을 이루었으며, 농업과 국방에도 많은 관심을 기울였습니다. 또한 교육과 문화 발전에 힘써서 조선시대 중흥기를 이끌었습니다.<|endoftext|>
